# __PyCity Schools__

In [669]:
#import python libraries
import pandas as pd

#specifying data
schooldata_file = "Resources/schools_complete.csv"
studentdata_file = "Resources/students_complete.csv"

#reading files into pandas
school_data = pd.read_csv(schooldata_file)
student_data = pd.read_csv(studentdata_file)

#combining data
pycity_data = pd.merge(student_data, school_data, how="left", on=["school_name","school_name"])

## _District Summary_

In [670]:
#calculate total number of schools, numbers, and budget
school_count = len(pycity_data["School ID"].unique())
student_count = len(pycity_data["Student ID"].unique())
district_budget = school_data["budget"].sum()

#calculate average math and reading score
averagedistrict_math = pycity_data["math_score"].mean()
averagedistrict_read = pycity_data["reading_score"].mean()

#calculate percentage of students passing math scores, passing reading scores, and both
passingmath = (pycity_data["math_score"] >= 70).sum()/student_count
passingreading = (pycity_data["reading_score"] >= 70).sum()/student_count
passingoverall = ((pycity_data["math_score"] >= 70) & (pycity_data["reading_score"] >= 70)).sum() / student_count

In [671]:
#create district summary table
district_summary = pd.DataFrame([
    {"Total Schools": school_count,
     "Total Students": student_count,
     "Total Budget": district_budget,
     "Average Math Score": averagedistrict_math,
     "Average Reading Score": averagedistrcit_read,
     "% Passing Math": passingmath,
     "% Passing Reading": passingreading,
     "% Overall Passing": passingoverall}
])

In [675]:
#format district summary table
district_dict = {'Total Students': '{0:,.0f}', 'Total Budget':'${0:,.2f}',
               '% Passing Math': '{:%}', '% Passing Reading': '{:%}', '% Overall Passing': '{:%}'}
district_summary.style.format(district_dict).hide_index()

Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
15,"39,170","$24,649,428.00",78.985371,81.877840,74.980853%,85.805463%,65.172326%


## _School Summary_

In [646]:
#create dataframe for school summary analysis
school_summary = pycity_data.groupby('school_name').agg({
    'type':['max'],
    'Student ID':['count'],
    'budget':['max'],
    'math_score':['mean'],
    'reading_score':['mean']})

#rename column titles and create new datasets
school_summary.columns = ["School Type", "Total Students", "Total School Budget", "Average Math Score", "Average Reading Score"]

#calculate the school budget per student column
school_summary["Per Student Budget"] = school_summary["Total School Budget"]/school_summary["Total Students"]

#pre-calculation for percentage of students passing math scores, passing reading scores, and both by school
passingmath_school = pycity_data.loc[pycity_data['math_score'] >= 70].groupby('school_name').count()['size']
passingread_school = pycity_data.loc[pycity_data['reading_score'] >= 70].groupby('school_name').count()['size']
passingoverall_school = pycity_data.loc[
    (pycity_data['math_score'] >= 70)
    &(pycity_data['reading_score'] >= 70)
    ].groupby('school_name').count()['size']

#calculate for percentage of students passing math scores, passing reading scores, and both by school
school_summary["% Passing Math"] = passingmath_school / school_summary["Total Students"]*100
school_summary["% Passing Reading"] = passingread_school / school_summary["Total Students"]*100
school_summary["% Overall Passing"] = passingoverall_school / school_summary["Total Students"]*100

In [647]:
#rearrange columns
school_summary = school_summary[['School Type', 'Total Students', 'Total School Budget','Per Student Budget',
       'Average Math Score', 'Average Reading Score', 
       '% Passing Math', '% Passing Reading', '% Overall Passing']]

#rename index column
school_summary.index.name= "School Name"

#format school summary table
school_dict = {'Total Students': '{0:,.0f}', 'Total School Budget':'${0:,.2f}',
               'Per Student Budget':'${0:,.2f}'}
school_summary.style.format(school_dict)

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
School Name,,,,,,,,,
Bailey High School,District,"4,976","$3,124,928.00",$628.00,77.048432,81.033963,66.680064,81.933280,54.642283
Cabrera High School,Charter,"1,858","$1,081,356.00",$582.00,83.061895,83.975780,94.133477,97.039828,91.334769
Figueroa High School,District,"2,949","$1,884,411.00",$639.00,76.711767,81.158020,65.988471,80.739234,53.204476
Ford High School,District,"2,739","$1,763,916.00",$644.00,77.102592,80.746258,68.309602,79.299014,54.289887
Griffin High School,Charter,"1,468","$917,500.00",$625.00,83.351499,83.816757,93.392371,97.138965,90.599455
Hernandez High School,District,"4,635","$3,022,020.00",$652.00,77.289752,80.934412,66.752967,80.862999,53.527508
Holden High School,Charter,427,"$248,087.00",$581.00,83.803279,83.814988,92.505855,96.252927,89.227166
Huang High School,District,"2,917","$1,910,635.00",$655.00,76.629414,81.182722,65.683922,81.316421,53.513884
Johnson High School,District,"4,761","$3,094,650.00",$650.00,77.072464,80.966394,66.057551,81.222432,53.539172


## _Top Performing Schools (By % Overall Passing)_

In [648]:
#find and sort the top five performing schools by % overall passing
topschools = school_summary.sort_values(by='% Overall Passing', ascending=False).head(5)

In [649]:
#format top school summary table
topschool_dict = {'Total Students': '{0:,.0f}', 'Total School Budget':'${0:,.2f}',
               'Per Student Budget':'${0:,.2f}'}
topschools.style.format(topschool_dict)

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
School Name,,,,,,,,,
Cabrera High School,Charter,"1,858","$1,081,356.00",$582.00,83.061895,83.975780,94.133477,97.039828,91.334769
Thomas High School,Charter,"1,635","$1,043,130.00",$638.00,83.418349,83.848930,93.272171,97.308869,90.948012
Griffin High School,Charter,"1,468","$917,500.00",$625.00,83.351499,83.816757,93.392371,97.138965,90.599455
Wilson High School,Charter,"2,283","$1,319,574.00",$578.00,83.274201,83.989488,93.867718,96.539641,90.582567
Pena High School,Charter,962,"$585,858.00",$609.00,83.839917,84.044699,94.594595,95.945946,90.540541


## _Bottom Performing Schools (By % Overall Passing)_

In [650]:
#find and sort the top five worst performing schools by % overall passing
worstschools = school_summary.sort_values(by=['% Overall Passing']).head(5)

In [651]:
#format worst school summary table
worstschool_dict = {'Total Students': '{0:,.0f}', 'Total School Budget':'${0:,.2f}',
               'Per Student Budget':'${0:,.2f}'}
worstschools.style.format(worstschool_dict)

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
School Name,,,,,,,,,
Rodriguez High School,District,"3,999","$2,547,363.00",$637.00,76.842711,80.744686,66.366592,80.220055,52.988247
Figueroa High School,District,"2,949","$1,884,411.00",$639.00,76.711767,81.158020,65.988471,80.739234,53.204476
Huang High School,District,"2,917","$1,910,635.00",$655.00,76.629414,81.182722,65.683922,81.316421,53.513884
Hernandez High School,District,"4,635","$3,022,020.00",$652.00,77.289752,80.934412,66.752967,80.862999,53.527508
Johnson High School,District,"4,761","$3,094,650.00",$650.00,77.072464,80.966394,66.057551,81.222432,53.539172


## _Math Scores by Grade_

In [652]:
#filter data by school for an average math score by grade and school
mathgrade_nine = pycity_data[pycity_data['grade'] == "9th"].groupby('school_name').mean()['math_score']
mathgrade_ten = pycity_data[pycity_data['grade'] == "10th"].groupby('school_name').mean()['math_score']
mathgrade_eleven = pycity_data[pycity_data['grade'] == "11th"].groupby('school_name').mean()['math_score']
mathgrade_twelve = pycity_data[pycity_data['grade'] == "12th"].groupby('school_name').mean()['math_score']
mathgrade_total = pycity_data.groupby('school_name').mean()['math_score']

In [653]:
#create math scores table with columns titles and place filtered data
mathscores_grade = pd.DataFrame({"9th":mathgrade_nine,"10th":mathgrade_ten,"11th":mathgrade_eleven,"12th":mathgrade_twelve,"Overall":mathgrade_total})

In [654]:
#rename index column and print math scores by grade table
mathscores_grade.index.name= "School Name"
mathscores_grade

,9th,10th,11th,12th,Overall
School Name,,,,,
Bailey High School,77.083676,76.996772,77.515588,76.492218,77.048432
Cabrera High School,83.094697,83.154506,82.765560,83.277487,83.061895
Figueroa High School,76.403037,76.539974,76.884344,77.151369,76.711767
Ford High School,77.361345,77.672316,76.918058,76.179963,77.102592
Griffin High School,82.044010,84.229064,83.842105,83.356164,83.351499
Hernandez High School,77.438495,77.337408,77.136029,77.186567,77.289752
Holden High School,83.787402,83.429825,85.000000,82.855422,83.803279
Huang High School,77.027251,75.908735,76.446602,77.225641,76.629414
Johnson High School,77.187857,76.691117,77.491653,76.863248,77.072464


## _Reading Scores by Grade_

In [655]:
#filter data by school for an average reading score by grade and school
readgrade_nine = pycity_data[pycity_data['grade'] == "9th"].groupby('school_name').mean()['reading_score']
readgrade_ten = pycity_data[pycity_data['grade'] == "10th"].groupby('school_name').mean()['reading_score']
readgrade_eleven = pycity_data[pycity_data['grade'] == "11th"].groupby('school_name').mean()['reading_score']
readgrade_twelve = pycity_data[pycity_data['grade'] == "12th"].groupby('school_name').mean()['reading_score']
readgrade_total = pycity_data.groupby('school_name').mean()['reading_score']

In [656]:
#create reading scores table with columns titles and place filtered data
readscores_grade = pd.DataFrame({"9th":readgrade_nine,"10th":readgrade_ten,"11th":readgrade_eleven,"12th":readgrade_twelve,"Overall":readgrade_total})

In [657]:
#rename index column and print reading scores by grade table
readscores_grade.index.name= "School Name"
readscores_grade

,9th,10th,11th,12th,Overall
School Name,,,,,
Bailey High School,81.303155,80.907183,80.945643,80.912451,81.033963
Cabrera High School,83.676136,84.253219,83.788382,84.287958,83.975780
Figueroa High School,81.198598,81.408912,80.640339,81.384863,81.158020
Ford High School,80.632653,81.262712,80.403642,80.662338,80.746258
Griffin High School,83.369193,83.706897,84.288089,84.013699,83.816757
Hernandez High School,80.866860,80.660147,81.396140,80.857143,80.934412
Holden High School,83.677165,83.324561,83.815534,84.698795,83.814988
Huang High School,81.290284,81.512386,81.417476,80.305983,81.182722
Johnson High School,81.260714,80.773431,80.616027,81.227564,80.966394


## _Scores by School Spending_

In [658]:
#create custom ranges and labels for groupby based on school spending per student
studentspending = [0,584,629,644,675]
spendinglabels = ["<$584","$585-629","$630-644","$645-675"]

In [659]:
#pull per student budget data from summary table and place into ranges
schoolscores = school_summary["Spending Ranges (Per Student)"]=pd.cut(school_summary["Per Student Budget"], studentspending,labels=spendinglabels)

In [660]:
#calculate and place average math and reading scores, percentage of students passing math and reading, and percentage of students passing both math and reading
schoolscores_spending = school_summary.groupby("Spending Ranges (Per Student)").mean()

In [661]:
#configure table with relevant information
del schoolscores_spending['Total Students']
del schoolscores_spending['Total School Budget']
del schoolscores_spending['Per Student Budget']

In [662]:
#format scores by school spending summary table
schoolscorespending_dict = {'Average Math Score':'{0:,.2f}',
                            'Average Reading Score':'{0:,.2f}',
                            '% Passing Math':'{0:,.2f}',
                            '% Passing Reading':'{0:,.2f}',
                            '% Overall Passing':'{0:,.2f}'}
schoolscores_spending.style.format(schoolscorespending_dict)

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Spending Ranges (Per Student),,,,,
<$584,83.46,83.93,93.46,96.61,90.37
$585-629,81.90,83.16,87.13,92.72,81.42
$630-644,78.52,81.62,73.48,84.39,62.86
$645-675,77.00,81.03,66.16,81.13,53.53


## _Scores by School Size_

In [663]:
#create custom ranges and labels for groupby based on school size
schoolstudentsize = [0,1000,2000,5000]
sizelabels = ["Small (<1000)","Medium (1000-2000)","Large (2000-5000)"]

In [664]:
#pull school size data from summary table and place into ranges
schoolsize = school_summary["School Size"]=pd.cut(school_summary["Total Students"], schoolstudentsize,labels=sizelabels)

In [665]:
#calculate and place average math and reading scores, percentage of students passing math and reading, and percentage of students passing both math and reading
schoolscores_size = school_summary.groupby("School Size").mean()

In [666]:
#configure and print table with relevant information
del schoolscores_size['Total Students']
del schoolscores_size['Total School Budget']
del schoolscores_size['Per Student Budget']
schoolscores_size

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
School Size,,,,,
Small (<1000),83.821598,83.929843,93.550225,96.099437,89.883853
Medium (1000-2000),83.374684,83.864438,93.599695,96.790680,90.621535
Large (2000-5000),77.746417,81.344493,69.963361,82.766634,58.286003


## _Scores by School Type_

In [667]:
#calculate and place data based on school type
schoolscores_type = school_summary.groupby("School Type").mean()

In [668]:
#configure and print table with relevant information
del schoolscores_type['Total Students']
del schoolscores_type['Total School Budget']
del schoolscores_type['Per Student Budget']
schoolscores_type

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
School Type,,,,,
Charter,83.473852,83.896421,93.620830,96.586489,90.432244
District,76.956733,80.966636,66.548453,80.799062,53.672208
